In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from scipy.optimize import fmin_bfgs, check_grad
import glob, os

os.chdir('../core')
import stitching_ssid as ssid

os.chdir('../../../../pyRRHDLDS/core')
import ssm_scripts

import pprint

# Covariance matrix completion 
- $\mbox{cov}(y) = C C^\top$
- if cov($y$) has missing off-diagonal blocks, parts of $C$ are underdetermined (change of latent basis)
- Bishop et al. (2014) introduced a basic algorithm for rotating latent bases based on overlap
- Srini had the idea to just learn all chunks of C at the same time using gradient descent

In [ ]:

p,n = 20,4
k,l = 1,1
overlap_size = 4

# create subpopulations
sub_pops = (np.arange(0,12), np.arange(8,p))
print('sub_pops', sub_pops)

obs_idx, idx_grp = ssid.get_obs_index_groups(obs_scheme={'sub_pops': sub_pops,'obs_pops': (0,1)},p=p)
overlaps, overlap_grp, idx_overlap = ssid.get_obs_index_overlaps(idx_grp, sub_pops)
print('idx_grp:', idx_grp)
print('obs_idx:', obs_idx)
        
Om, Ovw, Ovc = ssid.comp_subpop_index_mats(sub_pops,idx_grp,overlap_grp,idx_overlap)    
    
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(Om,interpolation='none')
plt.title('Observation pattern')
plt.subplot(1,3,2)
plt.imshow(Ovw,interpolation='none')
plt.title('Overlap pattern')
plt.subplot(1,3,3)
plt.imshow(Ovc,interpolation='none')
plt.title('Cross-overlap pattern')
plt.show()

for rep in range(10):
    
    C_true = np.random.normal(size=(p,n))
    Q_true = C_true.dot(C_true.T)
    Q_obs = Q_true * np.asarray( Om, dtype=float)
    Q_sti = Q_true * np.asarray(~Om, dtype=float)
    A, Pi = np.eye(n), np.eye(n)
    
    C_0 = np.random.normal(size=(p,n))
        
    def f_i(C):        
        return ssid.f_l2_Hankel(C,A,Pi,k,l,[Q_obs], Om)*np.sum(Om)
    def g_i(C):
        return ssid.g_C_l2_Hankel(C,A,Pi,k,l,[Q_obs],idx_grp, obs_idx)
    
    
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, C_0.reshape(p*n,)))
    
    C_est = fmin_bfgs(f_i, C_0.reshape(p*n,), fprime=g_i, gtol=1e-20).reshape(p,n)
    

    print('final squared error on observed parts:', 
          ssid.f_l2_block(C_est, A, Q_obs, Om))
    print('final squared error on overlapping parts:', 
          ssid.f_l2_block(C_est, A, Q_obs, Ovw))
    print('final squared error on cross-overlapping parts:',
          ssid.f_l2_block(C_est, A, Q_obs, Ovc))
    print('final squared error on stitched parts:',
          ssid.f_l2_block(C_est, A, Q_sti,~Om))

    plt.figure(figsize=(18,12))
    plt.subplot(2,2,1)
    plt.imshow(C_0.dot(C_0.T),interpolation='none')
    plt.title('Initial matrix (C_0 C_0^T)')
    plt.subplot(2,2,3)
    plt.imshow(Q_true,interpolation='none')
    plt.title('True  matrix')
    plt.subplot(2,2,4)
    plt.imshow(C_est.dot(C_est.T),interpolation='none')
    plt.title('Estimated matrix')
    plt.subplot(2,2,2)
    plt.imshow(Q_true[Ovw].reshape(-1,overlap_size), interpolation='none')
    plt.title('overlaps (extracted)')
    plt.show()

    #_,s,_ = np.linalg.svd(Q_true[Ovw].reshape(-1,2)[:2,:])
    #print('singular values first overlap:', s)
    #_,s,_ = np.linalg.svd(Q_true[Ovw].reshape(-1,2)[2:,:])
    #print('singular values second overlap:', s)
    
    print('\n')
    print('\n')


# Hankel covariance matrix completion 
- $ H_{k,l} = (I_k \otimes C) H^{xx}_{k,l} (I_l \otimes C)^\top $
- $ H^{xx}_{k,l} = \left[\begin{array}{llll} A \Pi & A^2 \Pi & \ldots & A^l \Pi\\ A^2 \Pi & A^3 \Pi & \ldots & A^{l+1} \Pi\\ \vdots & \vdots & \ddots & \vdots \\ A^{k} \Pi & A^{k+1} \Pi & \ldots & A^{k+l-1} \Pi \end{array} \right] $
- if cov($x$) has missing off-diagonal blocks, parts of $C$ are underdetermined (change of latent basis)
- each block of the Hankel cov matrix $H_{k,l}$ exhibits the same structure of missing entries as does cov($y$.
- We can combine the overlaps of the $k \times l$ many blocks of $H_{k,l}$ when collecting constraints on the latent basis.
- We here first assume $A,\Pi$ to be known, and apply Srini's idea of joint gradient descent on the whole $C$ to $H_{k,l}$. Then we learn $A, \Pi$ with the other parameters fixed

# Derivative w.r.t. $C$

In [ ]:
p,n = 11,2
k,l = 3,3

# create subpopulations
sub_pops = (np.arange(0,6), np.arange(5,p))
print('sub_pops', sub_pops)

obs_idx, idx_grp = ssid.get_obs_index_groups(obs_scheme={'sub_pops': sub_pops,'obs_pops': (0,1)},p=p)
overlaps, overlap_grp, idx_overlap = ssid.get_obs_index_overlaps(idx_grp, sub_pops)
print('idx_grp:', idx_grp)
print('obs_idx:', obs_idx)
        
Om, Ovw, Ovc = ssid.comp_subpop_index_mats(sub_pops,idx_grp,overlap_grp,idx_overlap)    
    
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(Om,interpolation='none')
plt.title('Observation pattern')
plt.subplot(1,3,2)
plt.imshow(Ovw,interpolation='none')
plt.title('Overlap pattern')
plt.subplot(1,3,3)
plt.imshow(Ovc,interpolation='none')
plt.title('Cross-overlap pattern')
plt.show()

for rep in range(10):
    
    C_true = np.random.normal(size=(p,n))
    Pi     = np.random.normal(size=(n,n))/np.sqrt(n)
    Pi     = Pi.dot(Pi.T) #np.eye(n) 
    A      = np.random.normal(size=(n,n))    
    Qs = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A, kl_)
        Qs.append(C_true.dot(Akl.dot(Pi)).dot(C_true.T) *np.asarray( Om,dtype=int) )
    Qs_full = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A, kl_)
        Qs_full.append(C_true.dot(Akl.dot(Pi)).dot(C_true.T) )       
    H_true = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l)
    H_obs = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l, Om)
    H_obs[np.where(H_obs==0)] = np.nan
    H_sti = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l,~Om)
    
    C_0 = np.random.normal(size=(p,n))
        
    def f_i(C):        
        return ssid.f_l2_Hankel(C,A,Pi,k,l,Qs, Om)*np.sum(Om)*(k*l)
    def g_i(C):
        return ssid.g_C_l2_Hankel(C,A,Pi,k,l,Qs,idx_grp, obs_idx)
    
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, C_0.reshape(p*n,)))
    
    C_est = fmin_bfgs(f_i, C_0.reshape(p*n,), fprime=g_i, gtol=1e-20).reshape(p,n)    

    print('final squared error on observed parts:', 
          ssid.f_l2_Hankel(C_est,A,Pi,k,l,Qs, Om))
    print('final squared error on overlapping parts:', 
          ssid.f_l2_Hankel(C_est,A,Pi,k,l,Qs,Ovw))
    print('final squared error on cross-overlapping parts:',
          ssid.f_l2_Hankel(C_est,A,Pi,k,l,Qs,Ovc))
    print('final squared error on stitched parts:',
          ssid.f_l2_Hankel(C_est,A,Pi,k,l,Qs_full,~Om))
    
    plt.figure(figsize=(16,12))
    plt.subplot(2,2,1)
    tmp = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l,Om)
    tmp[np.where(tmp==0)] = np.nan
    plt.imshow(tmp,interpolation='none')
    plt.title('Given data matrix (C_true, masked)')
    plt.subplot(2,2,2)
    plt.imshow(ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l),interpolation='none')
    plt.title('True  matrix (C_true)')    
    plt.subplot(2,2,3)
    plt.imshow(ssid.yy_Hankel_cov_mat(C_0,A,Pi,k,l),interpolation='none')
    plt.title('Initial matrix (C_0)')
    plt.subplot(2,2,4)
    plt.imshow(ssid.yy_Hankel_cov_mat(C_est,A,Pi,k,l),interpolation='none')
    plt.title('Estimated matrix (C_est)')
    plt.show()
    
    # closely compare stitched blocks:
    """
    plt.figure(figsize=(20,6))
    plt.subplot(1,3,1)
    tmpt = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l,~Om)
    tmpt = tmpt[np.where(tmpt != 0)].reshape(k*6,l*3)
    plt.imshow(tmpt,interpolation='none')
    plt.title('est. stitched Hankel subblocks (assembled)')

    plt.subplot(1,3,2)
    tmpe = ssid.yy_Hankel_cov_mat(C_est, A,Pi,k,l,~Om)
    tmpe = tmpe[np.where(tmpe != 0)].reshape(k*6,l*3)
    plt.imshow(tmpe,interpolation='none')
    plt.title('true stitched Hankel subblocks (assembled)')
    
    plt.subplot(1,3,3)
    plt.plot(tmpt, tmpe, 'r.')
    plt.xlabel('true')
    plt.ylabel('est')

    plt.show()    
    """

    #if np.sum(Ovw) > 0:
    #    _,s,_ = np.linalg.svd(Qs[0][Ovw].reshape(-1,1)[:2,:])
    #    print('singular values first overlap:', s)
    #    #_,s,_ = np.linalg.svd(Q_obs[Ovw].reshape(-1,2)[2:,:])
    #    #print('singular values second overlap:', s)    
    

# Derivative w.r.t. $\Pi$

In [ ]:
p,n = 10,4
k,l = 2,2

# create subpopulations
sub_pops = (np.arange(0,p//2), np.arange(p//2,p))
print('sub_pops', sub_pops)

obs_idx, idx_grp = ssid.get_obs_index_groups(obs_scheme={'sub_pops': sub_pops,'obs_pops': (0,1)},p=p)
overlaps, overlap_grp, idx_overlap = ssid.get_obs_index_overlaps(idx_grp, sub_pops)
print('idx_grp:', idx_grp)
print('obs_idx:', obs_idx)
        
Om, Ovw, Ovc = ssid.comp_subpop_index_mats(sub_pops,idx_grp,overlap_grp,idx_overlap)    
    
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(Om,interpolation='none')
plt.title('Observation pattern')
plt.subplot(1,3,2)
plt.imshow(Ovw,interpolation='none')
plt.title('Overlap pattern')
plt.subplot(1,3,3)
plt.imshow(Ovc,interpolation='none')
plt.title('Cross-overlap pattern')
plt.show()

for rep in range(5):
    
    C      = np.random.normal(size=(p,n))
    B_true = np.random.normal(size=(n,n))/np.sqrt(n)
    Pi_true = B_true.dot(B_true.T) #np.eye(n) 
    A      = np.random.normal(size=(n,n)) # np.diag(np.linspace(0.5, 0.9, n)) #
    Qs = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A, kl_)
        Qs.append(C.dot(Akl.dot(Pi_true)).dot(C.T) *np.asarray( Om,dtype=int) )
    Qs_full = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A, kl_)
        Qs_full.append(C.dot(Akl.dot(Pi_true)).dot(C.T) )       
    H_true = ssid.yy_Hankel_cov_mat(C,A,Pi_true,k,l)
    H_obs = ssid.yy_Hankel_cov_mat(C,A,Pi_true,k,l, Om)
    H_obs[np.where(H_obs==0)] = np.nan
    H_sti = ssid.yy_Hankel_cov_mat(C,A,Pi_true,k,l,~Om)
    
    B_0  = np.random.normal(size=(n,n))
    Pi_0 = B_0.dot(B_0.T)
    
    def f_i(B):        
        if len(B.shape)<2:
            B = B.reshape(A.shape[0], A.shape[0])
        return ssid.f_l2_Hankel(C,A,B.dot(B.T),k,l,Qs, Om)*np.sum(Om)*(k*l)
    def g_i(B):
        return ssid.g_B_l2_Hankel(C,A,B,k,l,Qs,Om)
    
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, B_0.reshape(n*n,)))
    
    B_est = fmin_bfgs(f_i, B_0.reshape(n*n,), fprime=g_i, gtol=1e-20).reshape(n,n)    
    Pi_est = B_est.dot(B_est.T)
    
    print('final squared error on observed parts:', 
          ssid.f_l2_Hankel(C,A,Pi_est,k,l,Qs, Om))
    print('final squared error on overlapping parts:', 
          ssid.f_l2_Hankel(C,A,Pi_est,k,l,Qs,Ovw))
    print('final squared error on cross-overlapping parts:',
          ssid.f_l2_Hankel(C,A,Pi_est,k,l,Qs,Ovc))
    print('final squared error on stitched parts:',
          ssid.f_l2_Hankel(C,A,Pi_est,k,l,Qs_full,~Om))
    
    plt.figure(figsize=(16,12))
    plt.subplot(2,2,1)
    tmp = ssid.yy_Hankel_cov_mat(C,A,Pi_true,k,l,Om)
    tmp[np.where(tmp==0)] = np.nan
    plt.imshow(tmp,interpolation='none')
    plt.title('Given data matrix (Pi_true, masked)')
    plt.subplot(2,2,2)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A,Pi_true,k,l),interpolation='none')
    plt.title('True  matrix (Pi_true)')    
    plt.subplot(2,2,3)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A,Pi_0,k,l),interpolation='none')
    plt.title('Initial matrix (Pi_0)')
    plt.subplot(2,2,4)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A,Pi_est,k,l),interpolation='none')
    plt.title('Estimated matrix (Pi_est)')
    plt.show()
    
    plt.figure(figsize=(16,12))
    plt.subplot(1,3,1)
    plt.imshow(Pi_0,interpolation='none')
    plt.title('Pi init')
    plt.subplot(1,3,2)
    plt.imshow(Pi_est,interpolation='none')
    plt.title('Pi est')
    plt.subplot(1,3,3)
    plt.imshow(Pi_true,interpolation='none')
    plt.title('Pi true')
    plt.show()


# Derivative w.r.t. $A$

- This one is nasty: we need the derivative of $A^m$ w.r.t. $A$, i.e. $\frac{\delta{}A^m}{\delta{}A} \in \mathbb{R}^{n \times n \times n \times n}$

In [ ]:
p,n = 5,3
k,l = 2,2

# create subpopulations
sub_pops = (np.arange(0,p//2), np.arange(p//2,p))
print('sub_pops', sub_pops)

obs_idx, idx_grp = ssid.get_obs_index_groups(obs_scheme={'sub_pops': sub_pops,'obs_pops': (0,1)},p=p)
overlaps, overlap_grp, idx_overlap = ssid.get_obs_index_overlaps(idx_grp, sub_pops)
print('idx_grp:', idx_grp)
print('obs_idx:', obs_idx)
        
Om, Ovw, Ovc = ssid.comp_subpop_index_mats(sub_pops,idx_grp,overlap_grp,idx_overlap)    
    
plt.figure(figsize=(20,10))
plt.subplot(1,3,1)
plt.imshow(Om,interpolation='none')
plt.title('Observation pattern')
plt.subplot(1,3,2)
plt.imshow(Ovw,interpolation='none')
plt.title('Overlap pattern')
plt.subplot(1,3,3)
plt.imshow(Ovc,interpolation='none')
plt.title('Cross-overlap pattern')
plt.show()

for rep in range(3):
    
    C      = np.random.normal(size=(p,n))
    B      = np.random.normal(size=(n,n))/np.sqrt(n)
    Pi     = B.dot(B.T) #np.eye(n) 
    A_true = np.random.normal(size=(n,n)) # np.diag(np.linspace(0.5, 0.9, n)) #
    Qs = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A_true, kl_)
        Qs.append(C.dot(Akl.dot(Pi)).dot(C.T) *np.asarray( Om,dtype=int) )
    Qs_full = []
    for kl_ in range(1,k+l):
        Akl = np.linalg.matrix_power(A_true, kl_)
        Qs_full.append(C.dot(Akl.dot(Pi)).dot(C.T) )       
    H_true = ssid.yy_Hankel_cov_mat(C,A_true,Pi,k,l)
    H_obs = ssid.yy_Hankel_cov_mat( C,A_true,Pi,k,l, Om)
    H_obs[np.where(H_obs==0)] = np.nan
    H_sti = ssid.yy_Hankel_cov_mat( C,A_true,Pi,k,l,~Om)
    
    A_0  = np.random.normal(size=(n,n))
    
    def f_i(A):        
        if len(A.shape)<2:
            A = A.reshape(Pi.shape[0], Pi.shape[0])
        return ssid.f_l2_Hankel(C,A,Pi,k,l,Qs, Om)*np.sum(Om)*(k*l)
    def g_i(A):
        return ssid.g_A_l2_Hankel(C,A,Pi,k,l,Qs,Om)
    
    
    print('difference in gradient to finite-differencing value:', check_grad(f_i, g_i, A_0.reshape(n*n,)))
    
    A_est = fmin_bfgs(f_i, A_0.reshape(n*n,), fprime=g_i, gtol=1e-20).reshape(n,n)    
    
    print('final squared error on observed parts:', 
          ssid.f_l2_Hankel(C,A_est,Pi,k,l,Qs, Om))
    print('final squared error on overlapping parts:', 
          ssid.f_l2_Hankel(C,A_est,Pi,k,l,Qs,Ovw))
    print('final squared error on cross-overlapping parts:',
          ssid.f_l2_Hankel(C,A_est,Pi,k,l,Qs,Ovc))
    print('final squared error on stitched parts:',
          ssid.f_l2_Hankel(C,A_est,Pi,k,l,Qs_full,~Om))
    
    plt.figure(figsize=(16,12))
    plt.subplot(2,2,1)
    tmp = ssid.yy_Hankel_cov_mat(C,A_true,Pi,k,l,Om)
    tmp[np.where(tmp==0)] = np.nan
    plt.imshow(tmp,interpolation='none')
    plt.title('Given data matrix (A_true, masked)')
    plt.subplot(2,2,2)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A_true,Pi,k,l),interpolation='none')
    plt.title('True  matrix (A_true)')    
    plt.subplot(2,2,3)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A_0,Pi,k,l),interpolation='none')
    plt.title('Initial matrix (A_0)')
    plt.subplot(2,2,4)
    plt.imshow(ssid.yy_Hankel_cov_mat(C,A_est,Pi,k,l),interpolation='none')
    plt.title('Estimated matrix (A_est)')
    plt.show()
    
    plt.figure(figsize=(16,12))
    plt.subplot(1,3,1)
    plt.imshow(A_0,interpolation='none')
    plt.title('A init')
    plt.subplot(1,3,2)
    plt.imshow(A_est,interpolation='none')
    plt.title('A est')
    plt.subplot(1,3,3)
    plt.imshow(A_true,interpolation='none')
    plt.title('A true')
    plt.show()
    

# Debug and unfinished stuff

In [ ]:
def collect_constraints(H, Ov, k, l, ovl_size):

    p = Om.shape[0]    
    assert Om.shape[1] == p and H.shape == (k*p, l*p)

    n_cnstr = np.sum(Ov)
    cnstr = 42 * np.ones(k*l*n_cnstr)


    for k_ in range(k):
        offset_k = k_*k*n_cnstr
        for l_ in range(l):
            cnstr[offset_k+l_*n_cnstr:offset_k+(l_+1)*n_cnstr] = H[k_*p:(k_+1)*p, l_*p:(l_+1)*p][Ov]

    return cnstr.reshape(-1,ovl_size)

ovl_size = 1
H = ssid.yy_Hankel_cov_mat(C_true,A,Pi,k,l)
x = collect_constraints(H, Ovw, k, l, ovl_size)
plt.imshow(x.reshape(k,l), interpolation='none')   
m,M = np.min(x), np.max(x)

plt.figure(figsize=(20,10))
for k_ in range(k):
    for l_ in range(l):
        plt.subplot(k,l,(k_*k)+l_)
        plt.imshow( x[ovl_size*(k_*k+l_):ovl_size*(k_*k+l_+1),:], interpolation='none', vmin=m, vmax=M)
        plt.title(str(x[ovl_size*(k_*k+l_):ovl_size*(k_*k+l_+1),:][0,0]))
plt.show()

tmp = x.reshape(k,l)
_,s,_ = np.linalg.svd(tmp)
print(s)

_,s,_ = np.linalg.svd(ssid.observability_mat({'A': A, 'C': C_true[3,:]},k))
print(s)

ssid.observability_mat({'A': A, 'C': C_true[3,:]},k)
